In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential

from tensorflow.keras.models import load_model
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt
import pydot
import graphviz

model = load_model('model.keras')

# Visualize the model architecture
plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)

# Display the model architecture plot
img = plt.imread('model.png')
plt.figure(figsize=(10, 10))
plt.imshow(img)
plt.axis('off')
plt.show()


AttributeError: module 'pydot' has no attribute 'InvocationException'